In [1]:
!pip install -q findspark

In [5]:
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable


In [26]:
!pip install pandas

In [6]:
!pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 35.3 MB 3.9 MB/s eta 0:00:01    |████▊                           | 5.2 MB 3.8 MB/s eta 0:00:08     |███████████████████████▍        | 25.8 MB 3.5 MB/s eta 0:00:03


In [18]:
pip install transformers

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 4.7 MB 5.0 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 2.5 MB/s eta 0:00:011
     |████████████████████████████████| 120 kB 7.1 MB/s eta 0:00:01
     |████████████████████████████████| 766 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 6.6 MB 6.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
Note: you may need to restart the kernel to use updated packages.


In [1]:
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads

model = AutoModelForPreTraining.from_pretrained('neuralmind/bert-base-portuguese-cased')
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)

/home/mt/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForPreTraining were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
from transformers import pipeline

pipe = pipeline('feature-extraction', model=model, tokenizer=tokenizer)

In [3]:
import findspark
findspark.init()


In [4]:
import pyspark
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("local[*]").getOrCreate()
print("Apache Spark version: ", spark.version)

Apache Spark version:  3.3.0


In [5]:
import pyspark.pandas as ps
import pandas as pd

In [6]:
sdf = ps.read_csv("semente.txt", header = None)

/home/mt/spark-3.3.0-bin-hadoop3/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [7]:
sdf.rename(columns = {0:"palavras"}, inplace = True)

In [109]:
from pyspark.ml.linalg import Vectors
import numpy as np

In [314]:
''''
emb_list1= [float(x) for x in pipe(sdf['palavras'].iloc[0])[0][0]]
emb_list2= [float(x) for x in pipe(sdf['palavras'].iloc[0])[0][1]]
emb_list3= [float(x) for x in pipe(sdf['palavras'].iloc[0])[0][2]]
emb_list4= [float(x) for x in pipe(sdf['palavras'].iloc[0])[0][3]]
emb_list = [(w+x+y+z)/4 for w, x, y, z in zip(emb_list1,emb_list2,emb_list3,emb_list4)]

embeddings = ps.DataFrame({'word': sdf['palavras'].iloc[0], 'word_embeddings': Vectors.dense(emb_list)}).to_spark()
'''

/home/mt/spark-3.3.0-bin-hadoop3/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [315]:
''''
pct = -10

for i, row in sdf.iterrows():
    if i >= 10:
        break
    if i%10 == 0:
        pct += 10
        print(pct, '%')
    emb = pipe(row['palavras'])
    if len(emb[0]) == 4:
        emb_list1= [float(x) for x in emb[0][0]]
        emb_list2= [float(x) for x in emb[0][1]]
        emb_list3= [float(x) for x in emb[0][2]]
        emb_list4= [float(x) for x in emb[0][3]]
        emb_list = [(w+x+y+z)/4 for w, x, y, z in zip(emb_list1,emb_list2,emb_list3,emb_list4)]
    else:
        emb_list1= [float(x) for x in emb[0][0]]
        emb_list2= [float(x) for x in emb[0][1]]
        emb_list3= [float(x) for x in emb[0][2]]
        emb_list = [(w+x+y)/4 for w, x, y in zip(emb_list1,emb_list2,emb_list3)]


    append_ps = ps.DataFrame({'word': row['palavras'], 'word_embeddings': Vectors.dense(emb_list)}).to_spark()
    embeddings = embeddings.union(append_ps)
    '''

0 %


In [309]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import VectorAssembler

In [317]:
'''
vectorAssembler = VectorAssembler(inputCols = ["word_embeddings"], outputCol = "features")
df = vectorAssembler.transform(embeddings)
'''

In [322]:
emb_list1= [float(x) for x in pipe(sdf['palavras'].iloc[0])[0][0]]
emb_list2= [float(x) for x in pipe(sdf['palavras'].iloc[0])[0][1]]
emb_list3= [float(x) for x in pipe(sdf['palavras'].iloc[0])[0][2]]
emb_list4= [float(x) for x in pipe(sdf['palavras'].iloc[0])[0][3]]
emb_list = [(w+x+y+z)/4 for w, x, y, z in zip(emb_list1,emb_list2,emb_list3,emb_list4)]

data = [(Vectors.dense(emb_list), 2.0)]

In [323]:
pct = -10

for i, row in sdf.iterrows():
    if i >= 10:
        break
    if i%10 == 0:
        pct += 10
        print(pct, '%')
    emb = pipe(row['palavras'])
    if len(emb[0]) == 4:
        emb_list1= [float(x) for x in emb[0][0]]
        emb_list2= [float(x) for x in emb[0][1]]
        emb_list3= [float(x) for x in emb[0][2]]
        emb_list4= [float(x) for x in emb[0][3]]
        emb_list = [(w+x+y+z)/4 for w, x, y, z in zip(emb_list1,emb_list2,emb_list3,emb_list4)]
    else:
        emb_list1= [float(x) for x in emb[0][0]]
        emb_list2= [float(x) for x in emb[0][1]]
        emb_list3= [float(x) for x in emb[0][2]]
        emb_list = [(w+x+y)/4 for w, x, y in zip(emb_list1,emb_list2,emb_list3)]
    
    data.append((Vectors.dense(emb_list), 2.0))

0 %


In [327]:
df = spark.createDataFrame(data, ["features", "weighCol"])
kmeans = KMeans().setK(5).setSeed(1)
cluster_model = kmeans.fit(df)

In [328]:
# Make predictions
predictions = cluster_model.transform(df)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

centers = cluster_model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Silhouette with squared euclidean distance = 0.45002824798918634
Cluster Centers: 
[-5.46598846 -5.6079756  -6.5021596  ... -5.66714466 -4.35651743
 -5.70500028]
[-7.0864588  -7.90321696 -7.70176697 ... -7.45909345 -6.67905378
 -8.0147984 ]
[-4.91280055 -4.84524731 -4.75558007 ... -5.05621576 -4.32895207
 -5.36712654]
[-5.63282096 -5.62252679 -5.38914275 ... -5.60925388 -4.91662037
 -5.95578969]
[-8.3905133  -7.91605306 -9.25838494 ... -9.29055715 -7.68108529
 -9.05738187]


In [326]:
model = kmeans.fit(df)